# PPE needs estimation from confirmed cases at the county level



As the number of covid cases in the states continues to grow, U.S. hospitals are already reporting shortages of key equipment needed to care for critically ill patients, including ventilators and personal protective equipment (PPE) for medical staff. 

![Screen Shot 2020-03-29 at 6 22 41 PM](https://user-images.githubusercontent.com/14057932/77913636-9c212200-7262-11ea-89ef-5d46e0db4b1b.png)

*Fig 1* PPE needs through confirmed convid19 case. 


### PPEs in this case include:

- Droplet mask/surgical mask;
- N95 mask
- Gloves
- Gown
- Googles
- BP cuff, stethoscope, pulse Ox, Ambu bug
- Sani-wipers

Besides PPE mentioned above we also include testkits and ventilators in the model. 



## Input

**The input data is the confirmed covid19 cases at county level** (Data from [USAFacts](https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/). We load this data when the notebook runs from [this url](https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv) using `covidcaremap.cases.get_usafacts_cases_by_county`. Note that running this notebook on different days will produce different results as it will use the latest data. We took in the newly added case daily for the PPE needs estimation. 

The PPEs are consumed through the covid19 patient flow that has shown in the above diagram. 

In order to make the estimation, we assumed:

- 14% test cases are tested as positive. This can change from county to county. 
- 80% of the confirmed positive cases will have very mild symptoms and only need home care; while
- 20% of the confirmed cases will end up in in-patient care; 
- 40% of patients from in-patient care may need critical care. 

- A patient may have about ten-room visits from doctors and nurses per day in in-patient care; but 
- A patient in critical care may need a room visit every 20 - 30 mins from doctors, nurses, other room visits may come from therapists, ventilator technicians. We assume 40 room visits per day for a critical care patient.

- Each patient need a ventilator in critical care;

- A patient may spend seven days in in-patient care;
- A patient may spend an average of 10 days at critical care. 



## Output

**County-level total PPE needs estimation from the current confirmed case from Jan 22nd to March 29th, 2020**

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from covidcaremap.cases import get_usafacts_cases_by_county

In [2]:
#only to use this geojson file because it has the county level geo-polygon
pop_geo = "../../data/processed/us_counties_with_pop.geojson"

In [41]:
# setting at one, so we can read in confirmed case daily from multiple sources
# in this workthough, we used data from https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/

confirmed_case = 1

# one health provider testing 10 patient perday
testing_ratio = 6

positive_rate = 0.14

inpatient_rate = 0.2
home_care_rate = 1 - inpatient_rate

homecare_rate = 1- inpatient_rate
test_cases = round(confirmed_case / positive_rate)

# 25% of in patient cases are going directly to ICU
# 75% of in patient cases will only need home care
dir_critical_rate = 0.25
support_care_rate = 0.75 

inpatient_care_days = 7
critical_care_days = 17

## patient to lab technician and healthcare workers
patient_tech_ratio = 170


# PPE use per patients per day from in patient to critical care
# room_entries: times / day
nurse_re = 6 
doc_re = 1.2
evs_re = 1
clinic_doc_re = 1
phle_re = 1.5

shift_per_day = 2

# patients to health provider ratio
p2nurse = 2
p2doc = 10
p2evs = 20
p2clinic = 10
p2phle = 10 

#ppe use by technician (ever 170 patiens)
ppe_use_tech = 6 

# means differently through the covid19 patient flow
double = 2


In [42]:
def ppes_ed_OB(patients, testing_ratio):
    """compute PPEs include (N95, gloves, gown, testkit, google, testkit and ventilator) 
    base on patients and healthcareworkers’ cosupution in ED or community testing 
    
    Args:
        patients (int): number of comfirmed cases

    Returns:
        total_ppes_ed_comm (dictionary): PPE needs for ED presentation or community testing
    """
    test_cases = round(patients / positive_rate)
    
    # every 30 patients were taked care by 5 healthe providers, see above diagram
    healthcareworkers = test_cases/testing_ratio
    droplet_masks = healthcareworkers 
    n95_needs =  healthcareworkers *double # double means two per day
    gloves = healthcareworkers *double # double means left and right hands
    testkits =  test_cases
    gowns =  healthcareworkers
    total_ppes_ed_comm = dict(N95s= n95_needs, Droplet_masks = droplet_masks, 
                              Gloves =gloves, Testkits = testkits, Gowns=gowns)
    
    return total_ppes_ed_comm


def ppes_lab_processing(patients, positive_rate): 
    """to compute the ppe burn during the lab processing
    Args:
        patients (int): number of total comfirmed cases from community test and ED presentation;
        
    Returns:
        total_ppes_lab (dictionary): PPE needs for lab processing
    """
    test_cases = round(patients / positive_rate)
     # each technician can run 170 test per day.
    technicians = test_cases/patient_tech_ratio 
    evs = test_cases/patient_tech_ratio

    droplet_masks = (technicians * ppe_use_tech +  evs ) 
    gloves =  (technicians * ppe_use_tech +  evs )  * double
    gowns =  technicians +  evs  


    total_ppes_lab = dict(Droplet_masks= droplet_masks,
                          Gloves= gloves, Gowns=gowns)

    return total_ppes_lab



def ppes_in_patient_care(patients, inpatient_care_days,inpatient_rate, support_care_rate):
    """to compute ppe needs during in patient care
    Args:
        patients (int): number of total patients from community test and ED presentation;
        in_patients_days(int): number of days each person will need to be in Patient Care
        
    Returns:
        total_ppes_in_patient (dictionary): PPE needs for in patient care
    """

    # 20% endup being positive and 20% of these in patient care
    in_patient_patients = patients * inpatient_rate * support_care_rate
    
    # number of health providers each day
    # can't use this to represent the health providers, cause we multiply the room entries
    nurses = in_patient_patients/p2nurse * shift_per_day * nurse_re
    docs = in_patient_patients/p2doc * shift_per_day * doc_re
    evss = in_patient_patients/p2evs * evs_re
    clinicians = in_patient_patients/p2doc * shift_per_day * clinic_doc_re
    phles = in_patient_patients/ p2phle * shift_per_day * phle_re
    
    #total health provider
    hps = sum([nurses, docs, clinicians, phles])
    
    
    gloves = (hps *double + evss * double ) * inpatient_care_days 
    droplet_masks = (hps  +  evss * double )* inpatient_care_days
    gowns = (hps +  evss * double )* inpatient_care_days 
    
    # goggle can be reused
    goggles = (hps +  evss * double )*double  
    
    bp_cuff = in_patient_patients # bp cuff, ambu bag can be reused
    wipers = in_patient_patients * inpatient_care_days 

    total_ppes_in_patient = dict(Gloves= gloves, Gowns=gowns, Droplet_masks= droplet_masks,
                                Goggles = goggles, BP_cuff = bp_cuff, Wiper = wipers)

    return total_ppes_in_patient


def ppes_critical_care(patients, critical_care_days, inpatient_rate, dir_critical_rate):
    """compute the ventilators and ppe needs based on the critical care patients
    Args:
        patients (int): number of total patients from community test and ED presentation;
        in_patients_days(int): number of days each person will need to be in Critical Care
        
    Returns:
        total_ppes_criticalcare (dictionary): PPE needs for in critical care
    """

    # 40% of in patient patients end up in critical care
    critical_care_patients = patients * inpatient_rate * dir_critical_rate
    
    ventilators = critical_care_patients
    
    # twice room entrences compares to in-patient care
    nurses = critical_care_patients/p2nurse * shift_per_day * nurse_re * double
    docs = critical_care_patients/p2doc * shift_per_day * doc_re  * double
    evss = critical_care_patients/p2evs * evs_re  * double
    clinicians = critical_care_patients/p2doc * shift_per_day * clinic_doc_re  * double
    phles = critical_care_patients/ p2phle * shift_per_day * phle_re * double
    
    hps = sum([nurses, docs, clinicians, phles])
   
    
    gloves = (hps + evss * double) * double *critical_care_days 
    droplet_masks = (hps +  evss * double )* critical_care_days 
    gowns = (hps +  evss * double) * critical_care_days 
    n95_needs = (hps +  evss * double )*double * critical_care_days
    
    # goggle can be reused
    goggles = (hps +  evss * double )*double 
    
    bp_cuff = critical_care_patients # bp cuff, ambu bag can be reused
    wipers = critical_care_patients * critical_care_days 

    total_ppes_criticalcare = dict(N95s = n95_needs, Gloves= gloves, Gowns=gowns, 
                                  Droplet_masks= droplet_masks, Goggles = goggles, 
                                  BP_cuff = bp_cuff, Wiper = wipers,
                                  Ventilators = ventilators)
    
    return total_ppes_criticalcare



In [43]:
# for every confirmed convid19 case
total_ppes_ed = ppes_ed_OB(confirmed_case, testing_ratio)
total_ppes_ed

{'N95s': 2.3333333333333335,
 'Droplet_masks': 1.1666666666666667,
 'Gloves': 2.3333333333333335,
 'Testkits': 7,
 'Gowns': 1.1666666666666667}

In [44]:
# for every confirmed convid19 case
total_ppes_lab = ppes_lab_processing(confirmed_case,  positive_rate)
total_ppes_lab

{'Droplet_masks': 0.2882352941176471,
 'Gloves': 0.5764705882352942,
 'Gowns': 0.08235294117647059}

In [45]:
# for every confirmed convid19 case
total_ppes_in_patient = ppes_in_patient_care(confirmed_case, inpatient_care_days, inpatient_rate, support_care_rate)
total_ppes_in_patient

{'Gloves': 14.259000000000002,
 'Gowns': 7.182,
 'Droplet_masks': 7.182,
 'Goggles': 2.052,
 'BP_cuff': 0.15000000000000002,
 'Wiper': 1.0500000000000003}

In [46]:
# for every confirmed convid19 case
total_ppes_criticalcare = ppes_critical_care(confirmed_case, critical_care_days, inpatient_rate, dir_critical_rate)
total_ppes_criticalcare

{'N95s': 23.256000000000007,
 'Gloves': 23.256000000000007,
 'Gowns': 11.628000000000004,
 'Droplet_masks': 11.628000000000004,
 'Goggles': 1.3680000000000003,
 'BP_cuff': 0.05,
 'Wiper': 0.8500000000000001,
 'Ventilators': 0.05}

In [47]:
# for every confirmed convid19 case from home in patient and critical cares 
testkits = total_ppes_ed['Testkits']
droplet_masks =  total_ppes_ed['Droplet_masks'] + total_ppes_lab['Droplet_masks'] + total_ppes_in_patient['Droplet_masks'] + total_ppes_criticalcare['Droplet_masks']
n95s = total_ppes_ed['N95s'] + total_ppes_criticalcare['N95s'] 
gloves = total_ppes_ed['Gloves']+ total_ppes_lab['Gloves'] + total_ppes_in_patient['Gloves'] + total_ppes_criticalcare['Gloves']  
gowns = total_ppes_ed['Gowns']+ total_ppes_lab['Gowns'] + total_ppes_in_patient['Gowns'] + total_ppes_criticalcare['Gowns'] 
goggles = total_ppes_in_patient['Goggles'] + total_ppes_criticalcare['Goggles']
bp_cuffs = total_ppes_in_patient['BP_cuff'] + total_ppes_criticalcare['BP_cuff']
wipers = total_ppes_in_patient['Wiper'] + total_ppes_criticalcare['Wiper']
ventilators = total_ppes_criticalcare['Ventilators']

In [48]:
total_ppes = round(sum([n95s, gloves, gowns,droplet_masks, goggles, bp_cuffs, wipers]))

In [49]:
# every confirm case may end up using 74 ppes, including n95 masks, gowns, faceshieldd, droplet_masks, googles, bp_cuff, wipers, ventilators
total_ppes

112

In [50]:
confirmed_cases = get_usafacts_cases_by_county()
confirmed_cases.head(4)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,1,4,6,6,6,6,7,7,10,10
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,4,4,5,5,10,15,18,19,23,25
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
def daily_cases_growth(df, opp_col):
    """ compute daily growth cases from the accumulated cases at county level
    Args:
        df (Pandas dataframe): the accumulated cases from USAFacts;
        opp_col (int):the column where cofirmed case starts to be recorded
    Returns:
        df(Pandas dataframe): dataframe that has daily growth confirmed cases
    """
    col_len = len(df.keys())
    cat_len = opp_col + 1
    if confirmed_cases.iloc[:, -1].dtype !='int64':
        confirmed_cases.iloc[:, -1] = confirmed_cases.iloc[:, -1].fillna(0)
        confirmed_cases.iloc[:, -1]= confirmed_cases.iloc[:, -1].astype('int64')
    # compute the daily growth instead of accumulated cases by county
    df.iloc[:, cat_len:]  = df.iloc[:, -(col_len-cat_len):] - df.shift(axis=1, fill_value=0).iloc[:, -(col_len-cat_len):]

    return df

In [52]:
def multiple_ppe(ppe_item_total, df_daily, opp_col, item_name):
    """to compute totoal ppes from given dates
    
    Args:
        ppe_item_total (int): the total ppe item from confirm covid19 cases;
        df (Pandas dataframe): data frame that records daily growning confirm cases 
        opp_col (int): the column where cofirmed case starts to be recorded
        
    Returns:
        ppes_df(Pandas dataframe): dataframe has the total sum of the ppe items
    """
    df_n = df_daily.apply(lambda x: x[opp_col:] * ppe_item_total, axis = 1) 
    total_g_ppes = df_n.apply(np.sum, axis =1)
    ppes_df = df_daily.iloc[:,:opp_col]
    # keep the nagative daily convid growth for ventilator compuation 
    if ppe_item_total == ventilators:
        total_g_ppes = np.floor(total_g_ppes)
    # take nagative daily gowth to be 0 for other ppe calculation
    else: 
        df_n= df_n.clip(lower = 0)
        total_g_ppes = df_n.apply(np.sum, axis =1)
    
    #if we see patient in the critical care, we assume at least 1 ventilator is needed
    ppes_df['total_cases'] = df_daily.iloc[:, opp_col:].sum(axis=1)
    ppes_df[item_name] = round(total_g_ppes)
    
    return ppes_df
    

In [53]:
df_daily_county = daily_cases_growth(confirmed_cases, 4)

In [54]:
df_daily_county.head()

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,1,3,2,0,0,0,1,0,3,0
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,1,0,1,0,5,5,3,1,4,2
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,2,1,0,1


In [55]:
total_ppes_df = multiple_ppe(total_ppes, df_daily_county, 4, "total_ppes")
total_ppes_df.head()

,countyFIPS,County Name,State,stateFIPS,total_cases,total_ppes
0,0,Statewide Unallocated,AL,1,0,0
1,1001,Autauga County,AL,1,10,1120
2,1003,Baldwin County,AL,1,25,2800
3,1005,Barbour County,AL,1,0,0
4,1007,Bibb County,AL,1,4,448


In [56]:
total_testkits = multiple_ppe(testkits, df_daily_county, 4, "testkits")
total_testkits.head()

,countyFIPS,County Name,State,stateFIPS,total_cases,testkits
0,0,Statewide Unallocated,AL,1,0,0
1,1001,Autauga County,AL,1,10,70
2,1003,Baldwin County,AL,1,25,175
3,1005,Barbour County,AL,1,0,0
4,1007,Bibb County,AL,1,4,28


In [57]:
total_testkits = multiple_ppe(testkits, confirmed_cases, 4, "testkits")
total_testkits.head()

,countyFIPS,County Name,State,stateFIPS,total_cases,testkits
0,0,Statewide Unallocated,AL,1,0,0
1,1001,Autauga County,AL,1,10,70
2,1003,Baldwin County,AL,1,25,175
3,1005,Barbour County,AL,1,0,0
4,1007,Bibb County,AL,1,4,28


In [58]:
total_droplet_masks = multiple_ppe(droplet_masks, df_daily_county, 4, "total_droplet_masks") 
total_droplet_masks.head()

,countyFIPS,County Name,State,stateFIPS,total_cases,total_droplet_masks
0,0,Statewide Unallocated,AL,1,0,0.0
1,1001,Autauga County,AL,1,10,203.0
2,1003,Baldwin County,AL,1,25,507.0
3,1005,Barbour County,AL,1,0,0.0
4,1007,Bibb County,AL,1,4,81.0


In [ ]:
total_n95s = multiple_ppe(n95s, df_daily_county, 4, "total_n95s") 
total_n95s.head()

In [ ]:
total_gloves = multiple_ppe(gloves, df_daily_county, 4, "total_gloves") 
total_gloves.head()

In [ ]:
total_gowns = multiple_ppe(gowns, df_daily_county, 4, "total_gowns") 
total_gowns.head()

In [ ]:
total_goggles = multiple_ppe(goggles, df_daily_county, 4, "total_goggles") 
total_goggles.head()

In [ ]:
total_bp_cuffs = multiple_ppe(bp_cuffs, confirmed_cases, 4, "total_bp_cuffs") 
total_bp_cuffs.head()

In [ ]:
total_wipers = multiple_ppe(wipers, confirmed_cases, 4, "total_wipers") 
total_wipers.head()

In [ ]:
total_ventilators = multiple_ppe(ventilators, confirmed_cases, 4, "total_ventilators") 
total_ventilators.head()

In [ ]:
total_ppes_df['total_cases'] =total_ventilators['total_cases']
total_ppes_df['testkits'] =total_testkits['testkits']
total_ppes_df['total_droplet_masks'] = total_droplet_masks['total_droplet_masks']
total_ppes_df['total_n95s'] = total_n95s['total_n95s']
total_ppes_df['total_gowns'] = total_gowns['total_gowns']
total_ppes_df['total_goggles'] = total_goggles['total_goggles']
total_ppes_df['total_bp_cuffs'] = total_bp_cuffs['total_bp_cuffs']
total_ppes_df['total_wipers'] = total_wipers['total_wipers']
total_ppes_df['total_ventilators'] = total_ventilators['total_ventilators']

In [29]:
total_ppes_df = total_ppes_df.rename(columns={'countyFIPS': 'COUNTY_FIPS'})
total_ppes_df.head()

,COUNTY_FIPS,County Name,State,stateFIPS,total_cases,total_ppes,testkits,total_droplet_masks,total_n95s,total_gowns,total_goggles,total_bp_cuffs,total_wipers,total_ventilators
0,0,Statewide Unallocated,AL,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1001,Autauga County,AL,1,10,1100,50,198.0,249.0,197.0,34.0,2.0,19.0,0.0
2,1003,Baldwin County,AL,1,25,2750,125,496.0,623.0,493.0,86.0,5.0,48.0,1.0
3,1005,Barbour County,AL,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1007,Bibb County,AL,1,4,440,20,79.0,100.0,79.0,14.0,1.0,8.0,0.0


In [40]:
sum(total_ppes_df['total_cases']), sum(total_ppes_df['testkits']), sum(total_ppes_df['total_n95s']), sum(total_ppes_df['total_ppes']), sum(total_ppes_df['total_ventilators']),sum(total_ppes_df['testkits'])

(242004, 1226205, 6112233.0, 26976510, 11391.0, 1226205)

In [31]:
gpd_df = gpd.read_file(pop_geo)
gpd_df.head()

,GEO_ID,STATE_FIPS,County Name,LSAD,CENSUSAREA,State,COUNTY_FIPS,Population,Population (20+),Population (65+),geometry
0,0500000US01001,01,Autauga,County,594.436,AL,01001,55601.0,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,0500000US01009,01,Blount,County,644.776,AL,01009,57840.0,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,0500000US01017,01,Chambers,County,596.531,AL,01017,33615.0,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,0500000US01021,01,Chilton,County,692.854,AL,01021,44153.0,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,0500000US01033,01,Colbert,County,592.619,AL,01033,54762.0,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."


In [32]:
len(gpd_df), len(total_ventilators)

(3221, 3195)

In [33]:
gpd_df['COUNTY_FIPS'].dtype, total_ppes_df['COUNTY_FIPS'].dtype

(dtype('O'), dtype('int64'))

In [34]:
gpd_df['COUNTY_FIPS'] =gpd_df['COUNTY_FIPS'].astype('int64')

In [35]:
gpd_df_new = gpd_df

In [36]:
gpd_df_new = gpd_df_new.drop(columns = ['LSAD', 'CENSUSAREA', 'Population'])
gpd_df_new.head()

,GEO_ID,STATE_FIPS,County Name,State,COUNTY_FIPS,Population (20+),Population (65+),geometry
0,0500000US01001,01,Autauga,AL,1001,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402..."
1,0500000US01009,01,Blount,AL,1009,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840..."
2,0500000US01017,01,Chambers,AL,1017,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772..."
3,0500000US01021,01,Chilton,AL,1021,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929..."
4,0500000US01033,01,Colbert,AL,1033,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587..."


In [37]:
total_ppes_geo = gpd_df_new.merge(total_ppes_df, on='COUNTY_FIPS', how='left')

In [38]:
total_ppes_geo = total_ppes_geo.drop(columns=['County Name_y', 'State_y', 'stateFIPS'])
total_ppes_geo = total_ppes_geo.rename(columns={'County Name_x': 'County Name',
                                               'State_x': 'State'})
total_ppes_geo.head()

,GEO_ID,STATE_FIPS,County Name,State,COUNTY_FIPS,Population (20+),Population (65+),geometry,total_cases,total_ppes,testkits,total_droplet_masks,total_n95s,total_gowns,total_goggles,total_bp_cuffs,total_wipers,total_ventilators
0,0500000US01001,01,Autauga,AL,1001,41177.0,8653.0,"POLYGON ((-86.49677 32.34444, -86.71790 32.402...",10.0,1100.0,50.0,198.0,249.0,197.0,34.0,2.0,19.0,0.0
1,0500000US01009,01,Blount,AL,1009,43175.0,10548.0,"POLYGON ((-86.57780 33.76532, -86.75914 33.840...",6.0,660.0,30.0,119.0,150.0,118.0,21.0,1.0,11.0,0.0
2,0500000US01017,01,Chambers,AL,1017,25934.0,6562.0,"POLYGON ((-85.18413 32.87053, -85.12342 32.772...",67.0,7370.0,335.0,1330.0,1670.0,1320.0,229.0,13.0,127.0,3.0
3,0500000US01021,01,Chilton,AL,1021,32683.0,7411.0,"POLYGON ((-86.51734 33.02057, -86.51596 32.929...",14.0,1540.0,70.0,278.0,349.0,276.0,48.0,3.0,27.0,0.0
4,0500000US01033,01,Colbert,AL,1033,42196.0,10914.0,"POLYGON ((-88.13999 34.58170, -88.13925 34.587...",5.0,550.0,25.0,99.0,125.0,99.0,17.0,1.0,10.0,0.0


In [39]:
total_ppes_geo.to_file("../../data/processed/ppe_needs_county_level.geojson", driver='GeoJSON')